# Define Environment & Library 

In [15]:
import os
import openai
import sys
import pandas as pd
sys.path.append('../..')

# Load environment
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# Set the OpenAI API key
openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
df = pd.read_csv("../data/12-08-data_faq.csv")
df = df.drop(columns=["id_faq", "nama_kategori"])

#  save to csv
df.to_csv("../data/clean_data_faq.csv", index=False)
df.head()

,question,answer
0,Kenapa diskon tidak muncul di detail produk?,Pastikan produk tersebut sedang promo atau hub...
1,"Kenapa Saat order stock produk tersedia, namun...",Sistem proses berdasarkan orderan yang masuk p...
2,"Saya tidak bisa masuk menu Order, muncul keter...",Pastikan status customer Anda aktif atau silah...
3,Bagaimana jika saya ingin bisa akses lebih dar...,Silakan menghubungi Salesman atau Supervisor a...
4,Kenapa saat saya log in muncul keterangan Ship...,Hal tersebut dikarenakan status Customer Anda ...


In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="../data/clean_data_faq.csv")
data = loader.load()

print(data)

[Document(metadata={'source': '../data/clean_data_faq.csv', 'row': 0}, page_content='question: Kenapa diskon tidak muncul di detail produk?\nanswer: Pastikan produk tersebut sedang promo atau hubungi WA EMOS Care'), Document(metadata={'source': '../data/clean_data_faq.csv', 'row': 1}, page_content='question: Kenapa Saat order stock produk tersedia, namun order di cancel karena stock tidak ada?\nanswer: Sistem proses berdasarkan orderan yang masuk pertama dan tergantung ketersedian stock saat orderan masuk sistem'), Document(metadata={'source': '../data/clean_data_faq.csv', 'row': 2}, page_content='question: Saya tidak bisa masuk menu Order, muncul keterangan Gagal ambil data\nanswer: Pastikan status customer Anda aktif atau silahkan menghubungi EMOS Care'), Document(metadata={'source': '../data/clean_data_faq.csv', 'row': 3}, page_content='question: Bagaimana jika saya ingin bisa akses lebih dari 1 Handphone?\nanswer: Silakan menghubungi Salesman atau Supervisor atau EMOS Care'), Docum

In [5]:
# text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, #max num of char 
    chunk_overlap=100,#overlap between chunks to maintain context 
    length_function=len,
    is_separator_regex=False,
)

chunk_docs = text_splitter.split_documents(data)

In [6]:
# define llm
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

/tmp/ipykernel_88037/290088060.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)


In [7]:
# Text embedding
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embedding_dim = 1536

# Connect to Milvus

In [ ]:
# Connect to database without creating new db 
from pymilvus import connections, db 
conn = connections.connect(
    host = "127.0.0.1",
    port = "19530",
    db_name = "my_database"
)

In [22]:
# list db 
db.list_database()

['default', 'my_database']

In [23]:

# use database 
db.using_database("my_database")

In [24]:
collection_name = "emos_collection"

In [25]:
import time 
from functools import lru_cache

def time_logged(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Time taken by {func.__name__}: {end_time - start_time: .4f} seconds") 
        return result 
    return wrapper       

# Connect to Redis 

In [26]:
import torch

torch.cuda.empty_cache()  # Clear PyTorch's cached memory

In [27]:
# Initialize Semantic Cache
from redisvl.extensions.llmcache import SemanticCache
llmcache = SemanticCache(
    name="llmcache",                     # underlying search index name
    redis_url="redis://localhost:6379",  # redis connection url string
    distance_threshold=0.1,              # semantic cache distance threshold
    overwrite=True
)

/home/kaniagalih/emos/emos-mobile-assistant/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
# look at the index specification created for the semantic cache lookup
!rvl index info -i llmcache

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Index Information:
╭──────────────┬────────────────┬──────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes     │ Index Options   │   Indexing │
├──────────────┼────────────────┼──────────────┼─────────────────┼────────────┤
│ llmcache     │ HASH           │ ['llmcache'] │ []              │          0 │
╰──────────────┴────────────────┴──────────────┴─────────────────┴────────────╯
Index Fields:
╭───────────────┬───────────────┬─────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────╮
│ Name          │ Attribute     │ Type    │ Field Option   │ Option Value   │ Field Option   │ Option Value   │ Field Option   │   Option Value │ Field Option    │ Option Value   │
├───────────────┼───────────────┼─────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┤
│ prompt        │ prom

In [ ]:

!HGETALL llmcache:a73030444569cbbb5b80cbf781c09467849349d29a4dd5ed720f2c4c407daa6e

/bin/bash: line 1: HGETALL: command not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# RAG Zero Shot Query Rewriting 

In [30]:
# retrieve using hybrid search 

from pymilvus import Collection, AnnSearchRequest, WeightedRanker

@time_logged
def retrieve_from_milvus(query, collection_name, k=5):
    # Embed the query 
    embedded_query = embeddings.embed_query(query)
    collection = Collection(name=collection_name)

    # annsearch for similarity search
    search_param = {
        "data": [embedded_query],
        "anns_field": "question_vector",
        "param": {
            "metric_type": "COSINE",
            "params": {"nprobe": 10} #will search in 10 cluster 
        },
        "limit": k
    }
    request = AnnSearchRequest(**search_param)

    # Perform the hybrid search in Milvus
    results = collection.hybrid_search(
        reqs=[request],
        rerank=WeightedRanker(0.8), #adding reranking startegy 
        limit=k,
        output_fields=["question", "answer"]  
    )

    # Extract the answers from search results
    answers = []
    if results:
        for hit in results[0]:
            answer = hit.entity.get('answer')
            if answer:
                answers.append(answer)

    return answers

In [31]:
# checking connection to milvus 
# Checking, manually search without LLM
from pprint import pprint

query = "How to change the password"
retrieved_docs = retrieve_from_milvus(query, collection_name, k=5)
pprint(retrieved_docs)

Time taken by retrieve_from_milvus:  7.7953 seconds
['1. Silakan masuk menu Profil lalu pilih ubah password\r\n'
 '2. Input Password baru\r\n'
 '3. Input kembali Password baru lalu klik Submit',
 '1. Silakan masuk menu Profil lalu pilih setting PIN\r\n'
 '2. Input PIN lama\r\n'
 '3. Input PIN baru lalu klik Submit',
 'Silakan klik forget password atau lupa password di Aplikasi.\r\n'
 'Pastikan nomer HP yang terdaftar aktif untuk mendapatkan Authentication Code',
 '1. Silakan masuk menu Poin atau Coin\r\n'
 '2. Klik lupa PIN\r\n'
 '3. Pastikan nomer HP yang terdaftar aktif untuk mendapatkan OTP',
 'Saat ini kami hanya memberikan akses 1 username 1 Handphone']


In [33]:
# Zero shot rewrite 

import re
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI


system_rewrite = """
You are a helpful assistant that generates multiple search queries based on a single input query.

Perform query expansion. If there are multiple common ways of phrasing a user question
or common synonyms for key words in the question, make sure to return multiple versions
of the query with the different phrasings.

If there are acronyms or words you are not familiar with, do not try to rephrase them.

Return 3 different versions of the question."""

@time_logged
def zero_shot(question):
    # Check if the zero-shot expanded questions are already cached
    cached_zero_shot = llmcache.check(prompt=f"zero_shot_{question}")
    
    if cached_zero_shot:
        # If the zero-shot output is cached, return it
        return cached_zero_shot[0]['response'].split("\n") 
    
    # If not in cache, generate the expanded queries using zero-shot
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_rewrite),
            ("human", "{context}")
        ]
    )
    
    zero_chain = LLMChain(llm=llm, prompt=prompt)
    
    expanded_queries = zero_chain.invoke({
        "context": question
    })
    
    # Split the response into multiple expanded questions
    expanded_question_list = expanded_queries["text"].split("\n")
    
    # Store the zero-shot expanded questions in the cache 
    llmcache.store(
        prompt=f"zero_shot_{question}",
        response="\n".join(expanded_question_list), 
        metadata={}
    )
    
    return expanded_question_list


In [34]:
# check the zero shot rewrite
question = "kenapa diskon tidak muncul di detail produk?"
result = zero_shot(question)

pprint(result)

/tmp/ipykernel_925/3514859553.py:38: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  zero_chain = LLMChain(llm=llm, prompt=prompt)


Time taken by zero_shot:  4.9482 seconds
['1. Mengapa diskon tidak terlihat di rincian produk?',
 '2. Apa alasan diskon tidak muncul pada detail produk?',
 '3. Kenapa potongan harga tidak tampil di informasi produk?']


In [35]:
# check the zero shot rewrite
question = "Mengapa diskon tidak terlihat di halaman detail produk??"
result = zero_shot(question)

pprint(result)

Time taken by zero_shot:  1.4346 seconds
['1. Mengapa saya tidak bisa melihat diskon di halaman detail produk?',
 '2. Kenapa diskon tidak muncul di halaman rincian produk?',
 '3. Apa alasan diskon tidak terlihat pada halaman detail barang?']


In [36]:
# check the zero shot rewrite
question = "Kenapa potongan harga tidak muncul pada detail produk?"
result = zero_shot(question)

pprint(result)

Time taken by zero_shot:  0.0978 seconds
['1. Mengapa saya tidak bisa melihat diskon di halaman detail produk?',
 '2. Kenapa diskon tidak muncul di halaman rincian produk?',
 '3. Apa alasan diskon tidak terlihat pada halaman detail barang?']


In [37]:
# new question
# check the zero shot rewrite
question = "Apakah ada perbedaan harga dan diskon jika order melalui emos?"
result = zero_shot(question)

pprint(result)

Time taken by zero_shot:  1.3499 seconds
['1. Apakah harga dan diskon berbeda jika melakukan pemesanan melalui emos?',
 '2. Adakah perbedaan dalam harga dan potongan jika saya memesan lewat emos?',
 '3. Bagaimana perbandingan harga dan diskon saat memesan melalui emos?']


In [38]:
import re
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory

answer_prompt = """
You are an intelligent system designed to assist users with their questions about the EMOS application and provide helpful, accurate, and natural-sounding responses. Follow ONLY the provided context to answer the question, thinking step-by-step before responding.
As "AI Assistant," your task is to provide clear and concise answers using the available context. If there is no relevant information, respond with "Hmm, I don't know the answer to that, its out of my resource" 
Refuse to answer any unrelated questions, never break character, and always base your response on the given context.
PAY ATTENTION if the user asks about products, or brands other than emos. Answer directly with "Hmm, I don't know the answer to that, its out of my resource"

Do not create an answer if:

- The answer cannot be determined from the context alone. In this case, respond with "Hmm, I don't know the answer to that, its out of my resource"
- The context is empty. Respond with "Hmm, I don't know the answer to that, its out of my resource"

Steps to answer the question:

1. Check if the user's question contains slang, short language, or uses synonyms. Interpret it accordingly.
2. Understand both the data source and the question.
3. Ensure the question relates to the data source. If it does not, respond with "Hmm, I don't know the answer to that, its out of my resource"
4. Read the entire question and provide an answer based on the context. If any word in the question does not match the data source, respond with "Hmm, I don't know the answer to that, its out of my resource"
    Example: "cara install aplikasi EMOS" IS NOT THE SAME AS "cara install aplikasi pembayaran."
    Example : "cara menggunakan point" IS NOT THE SAME AS "cara menggunakan point Tokopedia"
5. If the question relates to the data source, generate an answer based on both.
6. Verify the retrieved information before generating a response.
7. Do not provide explanations or steps unless the question directly relates to the data source.
8. Respond like an assistant without revealing your internal reasoning process.
9. Use bullet points if the response requires it.
10. If you know the user's name or prior history, address them accordingly, example: "Hi [user name]." or if you dont know the name just start the answer by "Hi There!"
11. If multiple users are involved, remember the last person who asked a question.

Answer in Indonesian. Keep your response clear, well-organized, and helpful.

Conversation format:

Question: {messages}
Answer: {context}

REMEMBER: If there is no relevant information within the context, just say "Hmm, I don't know the answer to that, its out of my resource". Don't try to make up an answer. Never break character.

"""
@time_logged
def generate_answer(query, context, message_history):
    # Initialize the question-answering prompt template
    question_answering_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", answer_prompt),  # Your answer prompt
            MessagesPlaceholder(variable_name="messages")
        ]
    )
    
    # Create the document chain for generating responses
    document_chain = LLMChain(llm=llm, prompt=question_answering_prompt)
    
    # Add the query to the message history
    message_history.add_message(HumanMessage(content=query))

    # Generate a response using the document chain
    response = document_chain.invoke({
        "messages": message_history.messages,  # Pass message history
        "context": context  # Use the retrieved context
    })

    return response["text"]


In [39]:
# To represent a document containing both page content and optional metadata
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata or {}

adding summarization function

In [40]:
summarization_prompt = """
Summarize the following responses into a concise and coherent answer. 
Focus on clarity and relevance, and remove any repetitive or unnecessary details.
Make sure that the generated output dont have redundancies 
provide output in a form that is easy to understand and if there are steps to follow make them in bullet point form.
check the rules in the answer prompt again so you don't violate them.
Anything that not having correlation with EMOS and data source, just Answer directly with "Hmm, I don't know the answer to that, its out of my resource"
Give the result using Bahasa Indonesia

Responses: {responses}
"""

# Create a chain for summarizing
@time_logged
def summarize_responses(combined_responses):
    # Define the summarization chain
    summarization_chain = LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_messages([("human", summarization_prompt)])
    )

    # Generate the summary by invoking the summarization chain
    summary = summarization_chain.invoke({
        "responses": combined_responses
    })
    
    return summary["text"]


In [41]:
@time_logged
def generate_response_zero_shot_with_summary(question):
    cached_response = llmcache.check(prompt=question)
    if cached_response:
        #found in cache
        return cached_response[0]['response']
    else:
        # Generate expanded questions using zero_shot 
        expanded_question = zero_shot(question)
        
        # Adding memory for conversation history
        message_history = ChatMessageHistory()
        
        all_responses = []
        
        for query in expanded_question:
            # Add each expanded query to the message history
            message_history.add_message(HumanMessage(content=query))
            
            # Retrieve the answer from a vector store (e.g., Milvus)
            answers = retrieve_from_milvus(query, collection_name="emos_collection", k=5)
            
            # Combine the retrieved answers as context
            docs = [Document(page_content=answer) for answer in answers]
            context = "\n".join([doc.page_content for doc in docs])
            
            # Generate a response using the document chain
            # response = document_chain.invoke({
            #     "messages": message_history.messages,
            #     "context": context
            # })
            
            response = generate_answer(query, context, message_history)
            
            # Collect each response
            # all_responses.append(response["text"])
            all_responses.append(response)
        
        # Concatenate all responses into a single string for summarization
        combined_responses = "\n".join(all_responses)
        
        # Summarize the combined responses
        # summary = summarization_chain.invoke({
        #     "responses": combined_responses
        # })
        
        summary = summarize_responses(combined_responses)
        
        llmcache.store(
            prompt=question,
            response=summary,
            metadata={}
        )
    
    return summary


In [42]:
import time
def test_semantic_cache(query):
    start_time = time.time()
    response = generate_response_zero_shot_with_summary(query)
    end_time = time.time()
    return response, end_time - start_time

In [43]:
question = "cara membeli barang di emos"
response =  generate_response_zero_shot_with_summary(question)


print(response)

Time taken by zero_shot:  1.4772 seconds
Time taken by retrieve_from_milvus:  0.7963 seconds
Time taken by generate_answer:  3.2840 seconds
Time taken by retrieve_from_milvus:  2.4417 seconds
Time taken by generate_answer:  2.6336 seconds
Time taken by retrieve_from_milvus:  0.3898 seconds
Time taken by generate_answer:  2.8063 seconds
Time taken by summarize_responses:  2.1049 seconds
Time taken by generate_response_zero_shot_with_summary:  16.0683 seconds
Untuk menggunakan aplikasi EMOS, ikuti langkah-langkah berikut:

1. **Download Aplikasi EMOS** di Google Play Store.
2. **Registrasi** dengan mengklik Sign Up.
3. **Pilih Distributor** yang melayani.
4. **Isikan data** sesuai kolom yang tersedia.
5. **Klik Register**.

Fitur yang tersedia di EMOS:
- Order kapan saja dan di mana saja.
- Cek ketersediaan stok produk.
- Cek status order secara langsung.
- Mendapatkan poin.
- Promo menarik untuk produk tertentu.

Untuk informasi lebih lanjut, silakan cek banner di halaman utama EMOS ata

In [44]:
question = "cara membeli barang di emos"
response =  generate_response_zero_shot_with_summary(question)


print(response)

Time taken by generate_response_zero_shot_with_summary:  0.0644 seconds
Untuk menggunakan aplikasi EMOS, ikuti langkah-langkah berikut:

1. **Download Aplikasi EMOS** di Google Play Store.
2. **Registrasi** dengan mengklik Sign Up.
3. **Pilih Distributor** yang melayani.
4. **Isikan data** sesuai kolom yang tersedia.
5. **Klik Register**.

Fitur yang tersedia di EMOS:
- Order kapan saja dan di mana saja.
- Cek ketersediaan stok produk.
- Cek status order secara langsung.
- Mendapatkan poin.
- Promo menarik untuk produk tertentu.

Untuk informasi lebih lanjut, silakan cek banner di halaman utama EMOS atau lihat profil distributor, karena kebijakan dapat berbeda-beda. Jika ada pertanyaan, Anda dapat menghubungi Salesman, Supervisor, atau EMOS Care.


In [45]:
question = "Gak bisa registrasi, munculnya ‘customer number not found’, itu kenapa ya?"
response =  generate_response_zero_shot_with_summary(question)


print(response)

Time taken by zero_shot:  1.4201 seconds
Time taken by retrieve_from_milvus:  0.5123 seconds
Time taken by generate_answer:  3.0018 seconds
Time taken by retrieve_from_milvus:  0.3642 seconds
Time taken by generate_answer:  1.9195 seconds
Time taken by retrieve_from_milvus:  0.5260 seconds
Time taken by generate_answer:  1.6542 seconds
Time taken by summarize_responses:  1.4418 seconds
Time taken by generate_response_zero_shot_with_summary:  11.0960 seconds
Untuk memastikan akses Anda ke EMOS, ikuti langkah-langkah berikut:

- Pastikan Anda adalah pelanggan Enseval dengan nomor pelanggan aktif.
- Verifikasi bahwa akun Anda telah diaktivasi oleh distributor.
- Cek keakuratan username, password, dan nomor pelanggan Anda.
- Pastikan jaringan internet Anda stabil.
- Pastikan status pelanggan Anda aktif dan tidak ada proses pembayaran yang tertunda.
- Periksa tidak ada masalah perizinan atau piutang yang overdue.
- Jika masih mengalami masalah, silakan hubungi EMOS Care untuk bantuan lebih 

In [46]:
question = "Gak bisa registrasi, munculnya ‘customer number not found’, itu kenapa ya?"
response =  generate_response_zero_shot_with_summary(question)


print(response)

Time taken by generate_response_zero_shot_with_summary:  0.0805 seconds
Untuk memastikan akses Anda ke EMOS, ikuti langkah-langkah berikut:

- Pastikan Anda adalah pelanggan Enseval dengan nomor pelanggan aktif.
- Verifikasi bahwa akun Anda telah diaktivasi oleh distributor.
- Cek keakuratan username, password, dan nomor pelanggan Anda.
- Pastikan jaringan internet Anda stabil.
- Pastikan status pelanggan Anda aktif dan tidak ada proses pembayaran yang tertunda.
- Periksa tidak ada masalah perizinan atau piutang yang overdue.
- Jika masih mengalami masalah, silakan hubungi EMOS Care untuk bantuan lebih lanjut. 

Catatan: Hanya satu username yang dapat digunakan untuk satu handphone.


In [47]:
# query
question = "cara ganti password gimana ya?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by zero_shot:  1.3967 seconds
Time taken by retrieve_from_milvus:  0.7238 seconds
Time taken by generate_answer:  0.8144 seconds
Time taken by retrieve_from_milvus:  0.5526 seconds
Time taken by generate_answer:  0.9499 seconds
Time taken by retrieve_from_milvus:  1.1300 seconds
Time taken by generate_answer:  1.2995 seconds
Time taken by summarize_responses:  1.2578 seconds
Time taken by generate_response_zero_shot_with_summary:  8.2539 seconds
Untuk mengubah password, ikuti langkah-langkah berikut:

1. Masuk ke menu Profil.
2. Pilih opsi "Ubah Password".
3. Masukkan Password baru.
4. Masukkan kembali Password baru dan klik "Submit".

Jika Anda lupa password, klik "Lupa Password" di aplikasi dan pastikan nomor HP yang terdaftar aktif untuk menerima Authentication Code.


In [48]:
# query
question = "cara ganti password gimana ya?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by generate_response_zero_shot_with_summary:  0.0648 seconds
Untuk mengubah password, ikuti langkah-langkah berikut:

1. Masuk ke menu Profil.
2. Pilih opsi "Ubah Password".
3. Masukkan Password baru.
4. Masukkan kembali Password baru dan klik "Submit".

Jika Anda lupa password, klik "Lupa Password" di aplikasi dan pastikan nomor HP yang terdaftar aktif untuk menerima Authentication Code.


In [49]:
question = "cara menukarkan point di tokopedia?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by zero_shot:  1.2086 seconds
Time taken by retrieve_from_milvus:  0.5211 seconds
Time taken by generate_answer:  0.6267 seconds
Time taken by retrieve_from_milvus:  0.6384 seconds
Time taken by generate_answer:  0.7308 seconds
Time taken by retrieve_from_milvus:  0.9452 seconds
Time taken by generate_answer:  0.7245 seconds
Time taken by summarize_responses:  0.6195 seconds
Time taken by generate_response_zero_shot_with_summary:  6.1493 seconds
Hmm, saya tidak tahu jawabannya, itu di luar sumber daya saya.


In [50]:
question = "halo min, koq orderan saya blm sampai, padahal sudah melakukan pemesanan seminggu yg lalu. Username saya:kiki_012"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by zero_shot:  2.5645 seconds
Time taken by retrieve_from_milvus:  0.7525 seconds
Time taken by generate_answer:  2.0721 seconds
Time taken by retrieve_from_milvus:  0.6991 seconds
Time taken by generate_answer:  2.4812 seconds
Time taken by retrieve_from_milvus:  2.3095 seconds
Time taken by generate_answer:  1.3222 seconds
Time taken by summarize_responses:  1.1665 seconds
Time taken by generate_response_zero_shot_with_summary:  13.5412 seconds
Untuk memeriksa status order Anda, ikuti langkah-langkah berikut:

- Masuk ke menu Status Order.
- Pastikan order tidak ter-cancel atau sudah mendapatkan Nomor Faktur.
- Periksa apakah order Anda sudah mendapatkan SJ (Surat Jalan).
- Pastikan Anda telah sampai pada tahap akhir dan mendapatkan konfirmasi "Order Sukses diproses."
- Pastikan jaringan internet Anda stabil dan status customer Anda aktif.


In [51]:
question = "halo min, koq orderan saya blm sampai, padahal sudah melakukan pemesanan seminggu yg lalu. Username saya:kiki_012"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by generate_response_zero_shot_with_summary:  0.0853 seconds
Untuk memeriksa status order Anda, ikuti langkah-langkah berikut:

- Masuk ke menu Status Order.
- Pastikan order tidak ter-cancel atau sudah mendapatkan Nomor Faktur.
- Periksa apakah order Anda sudah mendapatkan SJ (Surat Jalan).
- Pastikan Anda telah sampai pada tahap akhir dan mendapatkan konfirmasi "Order Sukses diproses."
- Pastikan jaringan internet Anda stabil dan status customer Anda aktif.


In [52]:
question = "cara install windows?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by zero_shot:  1.3517 seconds
Time taken by retrieve_from_milvus:  0.4639 seconds
Time taken by generate_answer:  0.8290 seconds
Time taken by retrieve_from_milvus:  0.5262 seconds
Time taken by generate_answer:  0.6443 seconds
Time taken by retrieve_from_milvus:  0.3286 seconds
Time taken by generate_answer:  0.7959 seconds
Time taken by summarize_responses:  0.8477 seconds
Time taken by generate_response_zero_shot_with_summary:  5.8890 seconds
Hmm, saya tidak tahu jawabannya, itu di luar sumber daya saya.


In [53]:
question = "cara menukarkan point di tokopedia?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by generate_response_zero_shot_with_summary:  0.0675 seconds
Hmm, saya tidak tahu jawabannya, itu di luar sumber daya saya.


In [54]:
question = "bagaimana cara bertransaksi dengan MosPay"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by zero_shot:  0.9424 seconds
Time taken by retrieve_from_milvus:  0.4767 seconds
Time taken by generate_answer:  1.9019 seconds
Time taken by retrieve_from_milvus:  0.4726 seconds
Time taken by generate_answer:  4.0821 seconds
Time taken by retrieve_from_milvus:  0.3375 seconds
Time taken by generate_answer:  2.2468 seconds
Time taken by summarize_responses:  4.0656 seconds
Time taken by generate_response_zero_shot_with_summary:  14.7065 seconds
Berikut adalah langkah-langkah untuk melakukan pembayaran melalui EMOS:

1. Masuk ke list Tagih.
2. Pilih faktur yang ingin dibayarkan, lalu klik Payment.
3. Pilih metode pembayaran yang diinginkan.
4. Salin Nomor Virtual Account yang muncul.
5. Gunakan Nomor Virtual Account untuk menyelesaikan pembayaran melalui M-Banking atau ATM.
6. Cek Instruksi Pembayaran di aplikasi setelah memilih metode pembayaran.

Catatan:
- Setiap pembayaran menggunakan MOSPay sebesar 300.000 (sebelum pajak) akan mendapatkan 1 coin, tidak berlaku kelipata

In [55]:
question = "cara menggunakan faktur CN bagaimana ya?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by zero_shot:  0.9175 seconds
Time taken by retrieve_from_milvus:  0.6638 seconds
Time taken by generate_answer:  0.7604 seconds
Time taken by retrieve_from_milvus:  0.9589 seconds
Time taken by generate_answer:  2.1363 seconds
Time taken by retrieve_from_milvus:  0.3398 seconds
Time taken by generate_answer:  1.7935 seconds
Time taken by summarize_responses:  2.2810 seconds
Time taken by generate_response_zero_shot_with_summary:  10.0345 seconds
Untuk melakukan pembayaran tagihan, ikuti langkah-langkah berikut:

1. Masuk ke list Tagihan.
2. Pilih invoice reguler dan CN yang ditandai dengan "-".
3. Klik Payment pada faktur yang ingin dibayarkan.
4. Pilih metode pembayaran yang diinginkan.
5. Salin Nomor Virtual Account yang muncul.
6. Gunakan Nomor Virtual Account untuk menyelesaikan pembayaran melalui M-Banking/ATM.
7. Cek Instruksi Pembayaran di aplikasi setelah memilih metode pembayaran.

Pastikan tidak ada masalah perizinan atau piutang yang overdue. Jika ada pertanyaan 

In [56]:
# question that has been ask in zero_shot 
question = "Apakah ada perbedaan harga dan diskon jika order melalui emos?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by generate_response_zero_shot_with_summary:  0.0670 seconds
1. Apakah harga dan diskon berbeda jika melakukan pemesanan melalui emos?
2. Adakah perbedaan dalam harga dan potongan jika saya memesan lewat emos?
3. Bagaimana perbandingan harga dan diskon saat memesan melalui emos?


In [57]:
# generated question from zero_shot 
question = "Apakah ada variasi harga dan diskon ketika order menggunakan emos?"
response = generate_response_zero_shot_with_summary(question)

print(response)

Time taken by zero_shot:  0.0777 seconds
Time taken by retrieve_from_milvus:  0.5889 seconds
Time taken by generate_answer:  0.9626 seconds
Time taken by retrieve_from_milvus:  0.4522 seconds
Time taken by generate_answer:  1.8976 seconds
Time taken by retrieve_from_milvus:  0.6424 seconds
Time taken by generate_answer:  1.5075 seconds
Time taken by summarize_responses:  1.7620 seconds
Time taken by generate_response_zero_shot_with_summary:  8.0558 seconds
Untuk informasi mengenai diskon produk reguler melalui EMOS, berikut adalah poin-poin penting:

- Diskon untuk produk reguler tetap sama jika melalui EMOS, Salesman, atau Call Center.
- Setiap Distributor memiliki kebijakan diskon yang berbeda. Silakan cek profil distributor untuk detail lebih lanjut.
- Untuk informasi lebih lanjut, hubungi Salesman, Supervisor, atau EMOS Care.

Fitur EMOS:
- Order bisa dilakukan kapan saja dan di mana saja.
- Cek ketersediaan stok produk.
- Cek status order secara langsung.
- Dapatkan poin.
- Promo 

NOTE & CONCLUSION 
- Zero shot rewriting just generate more question to make sure about the content in question given 
- System need to generate output from given question expantion and thats make the latency longer 
- Longest latency : 2m, shortest latency : 30s 
- The given output contains many redundancies 

SOLUTION
- Summarize the generated output 
- Latency : 14s

CONLUSION
- Generated output more details 
- But make the latency longer (14s) and shortest latency (8s)
- The generated output still inconsistent but better than without zero shot

# RAG Few Shot Query Rewriting 

In [60]:
# example query 
examples = [
    {
        "question": "Setelah melakukan order, cara untuk bayarnya gimana ya?",
        "answer": "Bagaimana cara melakukan pembayaran melalui MOSPay?",
    },
    {
        "question": "koq orderan saya blm sampai, padahal sudah melakukan pemesanan seminggu yg lalu?",
        "answer": "Kapan orderan dikirim?",
    },
    {
        "question": "Jika lupa username dan password, cara gantinya gimana ya?",
        "answer": "Bagaimana cara mengganti password?",
    }
    
]

# function for few shot question
def few_shot(question):
    
    example_prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "{question}"),
            ("ai", "{answer}")
        ]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )

    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_rewrite),
            few_shot_prompt,
            ("human", "{question}")
        ]
    )

    few_chain = LLMChain(llm=llm, prompt=final_prompt)

    expanded_queries = few_chain.invoke({
        "question": question
    })
    
    return expanded_queries["text"].split("\n")

In [61]:
# check the zero shot rewrite
question = "cara menukarkan point di tokopedia"
result = few_shot(question)

print(result)

['1. Bagaimana cara menukarkan poin di Tokopedia?', '2. Langkah-langkah untuk menukar poin di Tokopedia.', '3. Proses penukaran poin pada akun Tokopedia.']


In [62]:
# check the zero shot rewrite
question = "halo min, koq orderan saya blm sampai, padahal sudah melakukan pemesanan seminggu yg lalu. Username saya:kiki_012"
result = few_shot(question)

print(result)

['1. Kenapa orderan saya belum tiba meskipun sudah seminggu sejak pemesanan? Username saya: kiki_012', '2. Sudah seminggu sejak saya pesan, tapi orderan belum sampai. Apa yang harus saya lakukan? Username: kiki_012', '3. Order saya belum diterima meski sudah dipesan seminggu lalu. Bisa cek statusnya? Username saya: kiki_012']


In [63]:
def generate_response_few_shot_with_summary(question):
    cached_response = llmcache.check(prompt=question)
    if cached_response:
        #found in cache
        return cached_response[0]['response']
    else:
        # Generate expanded questions using zero_shot 
        expanded_question = few_shot(question)
        
    # Generate expanded questions using zero_shot 
    expanded_question = few_shot(question)
    
    # Adding memory for conversation history
    message_history = ChatMessageHistory()
    
    all_responses = []
    
    for query in expanded_question:
        # Add each expanded query to the message history
        message_history.add_message(HumanMessage(content=query))
        
        # Retrieve the answer from a vector store (e.g., Milvus)
        answers = retrieve_from_milvus(query, collection_name="emos_collection", k=5)
        
        # Combine the retrieved answers as context
        docs = [Document(page_content=answer) for answer in answers]
        context = "\n".join([doc.page_content for doc in docs])
        
        # Generate a response using the document chain
        response = document_chain.invoke({
            "messages": message_history.messages,
            "context": context
        })
        
        # Collect each response
        all_responses.append(response["text"])
    
    # Concatenate all responses into a single string for summarization
    combined_responses = "\n".join(all_responses)
    
    # Summarize the combined responses
    summary = summarization_chain.invoke({
        "responses": combined_responses
    })
    llmcache.store(
            prompt=question,
            response=summary["text"],
            metadata={}
        )
    
    return summary["text"]


In [64]:
# query
question = "cara ganti password gimana ya?"
response = generate_response_few_shot_with_summary(question)

print(response)

Untuk mengubah password, ikuti langkah-langkah berikut:

1. Masuk ke menu Profil.
2. Pilih opsi "Ubah Password."
3. Masukkan password baru.
4. Masukkan kembali password baru untuk konfirmasi.
5. Klik "Submit."

Jika Anda lupa password, klik "lupa password" di aplikasi dan pastikan nomor HP yang terdaftar aktif untuk menerima kode autentikasi.


In [65]:
# query
question = "cara ganti password gimana ya?"
response = generate_response_few_shot_with_summary(question)

print(response)

Untuk mengubah password, ikuti langkah-langkah berikut:

1. Masuk ke menu Profil.
2. Pilih opsi "Ubah Password."
3. Masukkan password baru.
4. Masukkan kembali password baru untuk konfirmasi.
5. Klik "Submit."

Jika Anda lupa password, klik "lupa password" di aplikasi dan pastikan nomor HP yang terdaftar aktif untuk menerima kode autentikasi.


In [66]:
question = "Gak bisa registrasi, munculnya ‘customer number not found’, itu kenapa ya?"
response =  generate_response_few_shot_with_summary(question)


print(response)

Untuk memastikan akses ke layanan EMOS, ikuti langkah-langkah berikut:

- Pastikan Anda adalah pelanggan Enseval dengan nomor pelanggan yang aktif.
- Verifikasi bahwa akun Anda telah diaktivasi oleh distributor mitra.
- Pastikan username, password, dan nomor pelanggan Anda benar.
- Periksa bahwa jaringan internet Anda stabil dan status pelanggan Anda aktif.
- Pastikan tidak ada masalah perizinan atau piutang yang belum dibayar.
- Hubungi EMOS Care untuk bantuan lebih lanjut atau jika ada masalah dengan status pelanggan yang sementara tidak aktif.
- Jika Anda mengganti perangkat atau memperbarui OS Android, hubungi Supervisor atau Salesman untuk pengecekan lebih lanjut.


In [67]:
question = "cara menukarkan point di tokopedia?"
response = generate_response_few_shot_with_summary(question)

print(response)

Hmm, saya tidak tahu jawabannya, itu di luar sumber daya saya.


In [68]:
question = "halo min, koq orderan saya blm sampai, padahal sudah melakukan pemesanan seminggu yg lalu. Username saya:kiki_012"
response = generate_response_few_shot_with_summary(question)

print(response)

- Masuk ke menu Status Order.
- Pastikan order tidak ter-cancel dan sudah mendapatkan Nomor Faktur.
- Pastikan order sudah mendapatkan SJ (Surat Jalan).
- Pastikan order Anda sudah sampai tahap akhir dengan konfirmasi Order Sukses diproses.
- Pastikan jaringan internet stabil dan status customer aktif.
- Jika masalah tetap terjadi meskipun status customer aktif, hubungi EMOS Care.


In [69]:
# query
question = "cara ganti password gimana ya?"
response = generate_response_few_shot_with_summary(question)

print(response)

Untuk mengubah password, ikuti langkah-langkah berikut:

1. Masuk ke menu Profil.
2. Pilih opsi "Ubah Password."
3. Masukkan password baru.
4. Masukkan kembali password baru untuk konfirmasi.
5. Klik "Submit."

Jika Anda lupa password, klik "lupa password" di aplikasi dan pastikan nomor HP yang terdaftar aktif untuk menerima kode autentikasi.


In [75]:
# query
question = "kenapa jaringan di hp saya tidak muncul"
response = generate_response_few_shot_with_summary(question)

print(response)

- Pastikan order Anda sudah diproses dan menjadi faktur dengan memeriksa menu Status Order.
- Masuk ke menu history redeem poin atau koin untuk memastikan statusnya sukses.
- Hubungi EMOS Care jika ada masalah dengan order atau faktur.
- Akses hanya diberikan untuk 1 username per 1 handphone.

Hmm, saya tidak tahu jawabannya, itu di luar sumber daya saya.


NOTE 
- Avg latency faster than zero shot
- after implement zero shot and comparing it with few shot, there is no significant difference 
- The generated output still inconsistent but better, just inconsistent in ambiguous question 

# Evaluation 

In [47]:
for datas in data:
    datas.metadata['filename'] = datas.metadata.get('source', '')

In [48]:
# generate test data
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import Document

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-4o")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
datates = generator.generate_with_langchain_docs(data, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})


d:\KALBE\EMA_CHATBOT\emos-mobile-assistant\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating: 100%|██████████| 10/10 [00:22<00:00,  2.30s/it]     


In [49]:
df = datates.to_pandas()
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,Why does the list of Virtual Accounts not appe...,[question: Kenapa list Virtual Account tidak m...,The answer to given question is not present in...,simple,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
1,How can you register for EMOS after downloadin...,[question: Bagaimana cara registrasi EMOS?\nan...,"After downloading the EMOS application, you ca...",simple,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
2,How can you cancel an order placed through EMO...,[question: Bagaimana cara meng-cancel orderan ...,You can cancel an order placed through EMOS by...,simple,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
3,Why does the message 'Shipto not active' appea...,[question: Kenapa saat saya log in muncul kete...,Hal tersebut dikarenakan status Customer Anda ...,simple,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
4,Apakah saya bisa menambahkan atau mengurangi p...,[question: Apakah saya bisa menambahkan atau m...,Untuk saat ini tidak bisa.,simple,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
5,What's restricted if order changes aren't allo...,[question: Apakah saya bisa menambahkan atau m...,The answer to given question is not present in...,reasoning,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
6,Who helps cancel an EMOS order and what do the...,[question: Bagaimana cara meng-cancel orderan ...,The answer to given question is not present in...,reasoning,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
7,How to regain access if login details are forg...,[question: Bagaimana jika saya lupa PIN?\nansw...,To regain access if login details are forgotte...,multi_context,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
8,What to do if customer status is inactive and ...,[question: Kenapa saat saya log in muncul kete...,If the customer status is inactive and order d...,multi_context,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True
9,Can an order be changed considering stock and ...,[question: Kenapa Saat order stock produk ters...,The answer to given question is not present in...,multi_context,[{'source': 'D:/KALBE/EMA_CHATBOT/emos-mobile-...,True


In [50]:
df = pd.DataFrame(df)
df.drop(columns=['evolution_type', 'metadata', 'episode_done'], inplace=True)
df

,question,contexts,ground_truth
0,Why does the list of Virtual Accounts not appe...,[question: Kenapa list Virtual Account tidak m...,The answer to given question is not present in...
1,How can you register for EMOS after downloadin...,[question: Bagaimana cara registrasi EMOS?\nan...,"After downloading the EMOS application, you ca..."
2,How can you cancel an order placed through EMO...,[question: Bagaimana cara meng-cancel orderan ...,You can cancel an order placed through EMOS by...
3,Why does the message 'Shipto not active' appea...,[question: Kenapa saat saya log in muncul kete...,Hal tersebut dikarenakan status Customer Anda ...
4,Apakah saya bisa menambahkan atau mengurangi p...,[question: Apakah saya bisa menambahkan atau m...,Untuk saat ini tidak bisa.
5,What's restricted if order changes aren't allo...,[question: Apakah saya bisa menambahkan atau m...,The answer to given question is not present in...
6,Who helps cancel an EMOS order and what do the...,[question: Bagaimana cara meng-cancel orderan ...,The answer to given question is not present in...
7,How to regain access if login details are forg...,[question: Bagaimana jika saya lupa PIN?\nansw...,To regain access if login details are forgotte...
8,What to do if customer status is inactive and ...,[question: Kenapa saat saya log in muncul kete...,If the customer status is inactive and order d...
9,Can an order be changed considering stock and ...,[question: Kenapa Saat order stock produk ters...,The answer to given question is not present in...


# Evaluation for zero shot 

In [51]:
# generate answer 
question = df['question'].tolist()
ground_truth = df["ground_truth"].tolist()
answers = []
contexts = []

for query in question:
    response = generate_response_zero_shot_with_summary(query) #change this with agent 
    answers.append(response)  
    retrieved_docs = retrieve_from_milvus(query, collection_name, k=5)
    contexts.append(retrieved_docs)

In [52]:
from datasets import Dataset 

data = {
    "question": df["question"].tolist(),
    "answer": answers,
    "contexts": contexts,
    "ground_truth": df["ground_truth"].tolist(),
}

dataset_zero = Dataset.from_dict(data)

print(dataset_zero)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 10
})


In [53]:
new_data_zero = dataset_zero.to_pandas()
new_data_zero

,question,answer,contexts,ground_truth
0,Why does the list of Virtual Accounts not appe...,1. Pastikan jaringan internet stabil dan gunak...,[Pastikan jaringan internet Anda dalam keadaan...,The answer to given question is not present in...
1,How can you register for EMOS after downloadin...,"Untuk menggunakan aplikasi EMOS, ikuti langkah...",[1. Download Aplikasi EMOS di Google Play Stor...,"After downloading the EMOS application, you ca..."
2,How can you cancel an order placed through EMO...,Silakan hubungi salah satu dari berikut ini:\n...,[Silakan menghubungi Salesman atau Supervisor ...,You can cancel an order placed through EMOS by...
3,Why does the message 'Shipto not active' appea...,Status Customer Anda sedang dinonaktifkan seme...,[Hal tersebut dikarenakan status Customer Anda...,Hal tersebut dikarenakan status Customer Anda ...
4,Apakah saya bisa menambahkan atau mengurangi p...,"Untuk saat ini, proses tidak bisa dilakukan. B...","[Untuk saat ini tidak bisa., Silahkan klik tom...",Untuk saat ini tidak bisa.
5,What's restricted if order changes aren't allo...,- Proses order berdasarkan urutan masuk dan ke...,[Sistem proses berdasarkan orderan yang masuk ...,The answer to given question is not present in...
6,Who helps cancel an EMOS order and what do the...,Silakan ikuti langkah berikut:\n\n- Hubungi Sa...,[Silakan menghubungi Salesman atau Supervisor ...,The answer to given question is not present in...
7,How to regain access if login details are forg...,"- Klik ""forget password"" atau ""lupa password"" ...",[Silakan klik forget password atau lupa passwo...,To regain access if login details are forgotte...
8,What to do if customer status is inactive and ...,Untuk memastikan pesanan Anda diproses dengan ...,[Silahkan klik tombol refresh yang ada di menu...,If the customer status is inactive and order d...
9,Can an order be changed considering stock and ...,- Sistem memproses pesanan berdasarkan urutan ...,"[Untuk saat ini tidak bisa., Sistem proses ber...",The answer to given question is not present in...


In [54]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [55]:
from ragas import evaluate

result = evaluate(
    dataset_zero,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating: 100%|██████████| 40/40 [00:22<00:00,  1.74it/s]


{'context_precision': 0.4583, 'faithfulness': 0.6785, 'answer_relevancy': 0.5714, 'context_recall': 0.8000}

In [56]:
df_eval = result.to_pandas()
df_eval

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,Why does the list of Virtual Accounts not appe...,1. Pastikan jaringan internet stabil dan gunak...,[Pastikan jaringan internet Anda dalam keadaan...,The answer to given question is not present in...,0.000000,0.666667,0.789716,1.0
1,How can you register for EMOS after downloadin...,"Untuk menggunakan aplikasi EMOS, ikuti langkah...",[1. Download Aplikasi EMOS di Google Play Stor...,"After downloading the EMOS application, you ca...",1.000000,0.866667,0.000000,1.0
2,How can you cancel an order placed through EMO...,Silakan hubungi salah satu dari berikut ini:\n...,[Silakan menghubungi Salesman atau Supervisor ...,You can cancel an order placed through EMOS by...,1.000000,0.000000,0.000000,0.0
3,Why does the message 'Shipto not active' appea...,Status Customer Anda sedang dinonaktifkan seme...,[Hal tersebut dikarenakan status Customer Anda...,Hal tersebut dikarenakan status Customer Anda ...,0.750000,1.000000,0.840395,1.0
4,Apakah saya bisa menambahkan atau mengurangi p...,"Untuk saat ini, proses tidak bisa dilakukan. B...","[Untuk saat ini tidak bisa., Silahkan klik tom...",Untuk saat ini tidak bisa.,1.000000,0.583333,0.000000,1.0
5,What's restricted if order changes aren't allo...,- Proses order berdasarkan urutan masuk dan ke...,[Sistem proses berdasarkan orderan yang masuk ...,The answer to given question is not present in...,0.000000,0.727273,0.813250,1.0
6,Who helps cancel an EMOS order and what do the...,Silakan ikuti langkah berikut:\n\n- Hubungi Sa...,[Silakan menghubungi Salesman atau Supervisor ...,The answer to given question is not present in...,0.000000,1.000000,0.768562,1.0
7,How to regain access if login details are forg...,"- Klik ""forget password"" atau ""lupa password"" ...",[Silakan klik forget password atau lupa passwo...,To regain access if login details are forgotte...,0.833333,0.500000,0.869051,1.0
8,What to do if customer status is inactive and ...,Untuk memastikan pesanan Anda diproses dengan ...,[Silahkan klik tombol refresh yang ada di menu...,If the customer status is inactive and order d...,0.000000,0.714286,0.792929,0.0
9,Can an order be changed considering stock and ...,- Sistem memproses pesanan berdasarkan urutan ...,"[Untuk saat ini tidak bisa., Sistem proses ber...",The answer to given question is not present in...,0.000000,0.727273,0.840074,1.0


# Evaluation for few shot

In [57]:
# generate answer 
question = df['question'].tolist()
ground_truth = df["ground_truth"].tolist()
answers = []
contexts = []

for query in question:
    response = generate_response_few_shot_with_summary(query) #change this with agent 
    answers.append(response)  
    retrieved_docs = retrieve_from_milvus(query, collection_name, k=5)
    contexts.append(retrieved_docs)

In [58]:
from datasets import Dataset 

data = {
    "question": df["question"].tolist(),
    "answer": answers,
    "contexts": contexts,
    "ground_truth": df["ground_truth"].tolist(),
}

dataset_few = Dataset.from_dict(data)

print(dataset_few)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 10
})


In [59]:
new_data_few = dataset_few.to_pandas()
new_data_few

,question,answer,contexts,ground_truth
0,Why does the list of Virtual Accounts not appe...,- Pastikan jaringan internet stabil dan gunaka...,[Pastikan jaringan internet Anda dalam keadaan...,The answer to given question is not present in...
1,How can you register for EMOS after downloadin...,- Unduh aplikasi EMOS di Google Play Store.\n-...,[1. Download Aplikasi EMOS di Google Play Stor...,"After downloading the EMOS application, you ca..."
2,How can you cancel an order placed through EMO...,Hubungi salah satu dari berikut ini untuk mend...,[Silakan menghubungi Salesman atau Supervisor ...,You can cancel an order placed through EMOS by...
3,Why does the message 'Shipto not active' appea...,- Pastikan status customer Anda aktif. Jika ti...,[Hal tersebut dikarenakan status Customer Anda...,Hal tersebut dikarenakan status Customer Anda ...
4,Apakah saya bisa menambahkan atau mengurangi p...,"- Saat ini, sistem tidak bisa memproses order....","[Untuk saat ini tidak bisa., Silahkan klik tom...",Untuk saat ini tidak bisa.
5,What's restricted if order changes aren't allo...,- Sistem memproses berdasarkan urutan order ya...,[Sistem proses berdasarkan orderan yang masuk ...,The answer to given question is not present in...
6,Who helps cancel an EMOS order and what do the...,Hubungi salah satu dari berikut ini untuk mend...,[Silakan menghubungi Salesman atau Supervisor ...,The answer to given question is not present in...
7,How to regain access if login details are forg...,"- Klik ""forget password"" atau ""lupa password"" ...",[Silakan klik forget password atau lupa passwo...,To regain access if login details are forgotte...
8,What to do if customer status is inactive and ...,Untuk memastikan orderan Anda diproses dengan ...,[Silahkan klik tombol refresh yang ada di menu...,If the customer status is inactive and order d...
9,Can an order be changed considering stock and ...,Untuk memastikan orderan Anda diproses dengan ...,"[Untuk saat ini tidak bisa., Sistem proses ber...",The answer to given question is not present in...


In [60]:
from ragas import evaluate

result = evaluate(
    dataset_few,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating: 100%|██████████| 40/40 [00:29<00:00,  1.35it/s]


{'context_precision': 0.4783, 'faithfulness': 0.7708, 'answer_relevancy': 0.5906, 'context_recall': 0.8000}

In [61]:
df_eval = result.to_pandas()
df_eval

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,Why does the list of Virtual Accounts not appe...,- Pastikan jaringan internet stabil dan gunaka...,[Pastikan jaringan internet Anda dalam keadaan...,The answer to given question is not present in...,0.000000,1.000000,0.000000,1.0
1,How can you register for EMOS after downloadin...,- Unduh aplikasi EMOS di Google Play Store.\n-...,[1. Download Aplikasi EMOS di Google Play Stor...,"After downloading the EMOS application, you ca...",1.000000,1.000000,0.946204,1.0
2,How can you cancel an order placed through EMO...,Hubungi salah satu dari berikut ini untuk mend...,[Silakan menghubungi Salesman atau Supervisor ...,You can cancel an order placed through EMOS by...,1.000000,NaN,0.000000,0.0
3,Why does the message 'Shipto not active' appea...,- Pastikan status customer Anda aktif. Jika ti...,[Hal tersebut dikarenakan status Customer Anda...,Hal tersebut dikarenakan status Customer Anda ...,0.750000,0.500000,0.866160,1.0
4,Apakah saya bisa menambahkan atau mengurangi p...,"- Saat ini, sistem tidak bisa memproses order....","[Untuk saat ini tidak bisa., Silahkan klik tom...",Untuk saat ini tidak bisa.,1.000000,0.625000,0.795195,1.0
5,What's restricted if order changes aren't allo...,- Sistem memproses berdasarkan urutan order ya...,[Sistem proses berdasarkan orderan yang masuk ...,The answer to given question is not present in...,0.000000,0.625000,0.788073,1.0
6,Who helps cancel an EMOS order and what do the...,Hubungi salah satu dari berikut ini untuk mend...,[Silakan menghubungi Salesman atau Supervisor ...,The answer to given question is not present in...,0.000000,NaN,0.000000,1.0
7,How to regain access if login details are forg...,"- Klik ""forget password"" atau ""lupa password"" ...",[Silakan klik forget password atau lupa passwo...,To regain access if login details are forgotte...,0.833333,1.000000,0.859013,1.0
8,What to do if customer status is inactive and ...,Untuk memastikan orderan Anda diproses dengan ...,[Silahkan klik tombol refresh yang ada di menu...,If the customer status is inactive and order d...,0.200000,0.750000,0.813145,0.0
9,Can an order be changed considering stock and ...,Untuk memastikan orderan Anda diproses dengan ...,"[Untuk saat ini tidak bisa., Sistem proses ber...",The answer to given question is not present in...,0.000000,0.666667,0.838329,1.0
